# Feature matrices optimization (cont.)

In this notebook, I'll try to optimize even more the computation of one feature
of the feature matrix used in the HSIC lasso algorithm.

For this notebook, I focus the test case on an input of 1 million samples.

I will focus on performances using CPU core, as the application is parallelized
over features on many cores.

In [1]:
import numpy as np
from pyHSICLasso.hsic_lasso import compute_kernel
from pyHSICLasso.kernel_tools import kernel_gaussian
from numba import njit, guvectorize

from hsic_optimization.benchmark import generate_data

In [2]:
X, _ = generate_data(1_000_000, 1, 1, seed=12345, transpose_X=True)
X.shape

(1, 1000000)

For comparison purpose, let's compute the Gaussian kernel feature with pyHSICLasso.

In [3]:
%%time
B = 20
M = 3

k_ref = compute_kernel(X, "Gaussian", B=B, M=M)
k_ref.shape

CPU times: user 13.9 s, sys: 70.4 ms, total: 13.9 s
Wall time: 13.9 s


(60000000,)

## Baseline function

First, let's check the performance of the currently used method (at the time of
writing).

In [4]:
@njit
def kernel_lookup_1(x, lookup):
    n = len(x)
    out = np.empty((n, n), dtype=lookup.dtype)
    for i in range(n):
        for j in range(i, n):
            res = lookup[x[i], x[j]]
            out[i, j] = res
            out[j, i] = res
    return out


def compute_kernel_1(x, lookup, B=0, M=1, discarded=0):
    n = len(x)
    H = np.eye(B, dtype=np.float32) - np.float32(1 / B)
    K = np.zeros(n * B * M, dtype=np.float32)

    st = 0
    ed = B**2
    index = np.arange(n)
    for m in range(M):
        np.random.seed(m)
        index = np.random.permutation(index)
        X_k = x[index]

        for i in range(0, n - discarded, B):
            j = min(n, i + B)

            k = kernel_lookup_1(X_k[i:j], lookup)
            k = (H @ k) @ H
            k = k / (np.sqrt(np.sum(k**2)) + 1e-9)

            K[st:ed] = k.ravel()
            st += B**2
            ed += B**2

    return K

The lookup table for the Gaussian kernel needs to be computed once per input
feature

In [5]:
arr = np.array([[0, 1, 2]])
x_std = X.std() + 1e-19
lookup = kernel_gaussian(arr / x_std, arr / x_std, 1.0).astype(np.float32)

In [6]:
k_1 = compute_kernel_1(X[0], lookup, B, M)
np.allclose(k_ref, k_1)

True

Unfortunately, the accelerated version has a different numerical precision, such
that the results are not matching anymore.

Actually, it looks like this version has a better numerical precision that the
reference code. More details are available in the
[Feature matrices optimization](03a_feature_optimization.ipynb) notebook.

In [7]:
compute_kernel_1b = njit(compute_kernel_1)
k_1b = compute_kernel_1b(X[0], lookup, B, M)
np.allclose(k_ref, k_1b)

True

In [8]:
%timeit compute_kernel_1b(X[0], lookup, B, M)

744 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


A first quick win could be to use the `fastmath` option, if this doesn't change
the results too much from the first accelerated version.

In [9]:
compute_kernel_1c = njit(fastmath=True)(compute_kernel_1)
k_1c = compute_kernel_1c(X[0], lookup, B, M)
np.allclose(k_1b, k_1c)

True

In [10]:
%timeit compute_kernel_1c(X[0], lookup, B, M)

730 ms ± 33.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


As the inputs are only 0, 1 and 2, we can also use a simpler datatype for X.

In [11]:
X2 = X.astype(np.int8)
k_1d = compute_kernel_1c(X2[0], lookup, B, M)
np.allclose(k_1b, k_1d)

True

In [12]:
%timeit compute_kernel_1c(X2[0], lookup, B, M)

693 ms ± 25.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
X3 = X.astype(np.uint8)
k_1e = compute_kernel_1c(X3[0], lookup, B, M)
np.allclose(k_1b, k_1e)

True

In [14]:
%timeit compute_kernel_1c(X3[0], lookup, B, M)

690 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Universal function

An other approach is to convert the core loop as a universal function and see
if this can improve the performances.

In [15]:
@guvectorize(
    ["u1[:], f4[:, :], f4[:, :]"], "(B),(N,N)->(B,B)", nopython=True, fastmath=True
)
def kernel_lookup_2(x, lookup, res):
    k = kernel_lookup_1(x, lookup)
    H = np.eye(B, dtype=np.float32) - np.float32(1 / B)
    k = (H @ k) @ H
    res[:] = k / (np.sqrt(np.sum(k**2)) + 1e-9)


def compute_kernel_2(x, lookup, B=0, M=1, discarded=0):
    K = []

    index = np.arange(len(x))
    for m in range(M):
        np.random.seed(m)
        index = np.random.permutation(index)
        X_k = x[index].reshape(-1, B).copy()
        K_k = kernel_lookup_2(X_k, lookup)
        K.append(K_k.ravel())

    K = np.concatenate(K)

    return K

In [16]:
k_2 = compute_kernel_2(X3[0], lookup, B, M)
np.allclose(k_1b, k_2)

True

In [17]:
%timeit compute_kernel_2(X3[0], lookup, B, M)

818 ms ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
def compute_kernel_3(x, lookup, B=0, M=1, discarded=0):
    n = len(x)
    K = np.zeros(n * B * M, dtype=np.float32)

    i = 0

    index = np.arange(len(x))
    for m in range(M):
        np.random.seed(m)
        index = np.random.permutation(index)
        X_k = x[index].reshape(-1, B).copy()

        K_k = kernel_lookup_2(X_k, lookup)
        K[i : i + n * B] = K_k.ravel()
        i += n * B

    return K

In [19]:
k_3 = compute_kernel_3(X3[0], lookup, B, M)
np.allclose(k_1b, k_3)

True

In [20]:
%timeit compute_kernel_3(X3[0], lookup, B, M)

798 ms ± 2.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Linear algebra libraries

Let's double check which version of BLAS is used.

In [21]:
np.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scale_wlg_persistent/filesets/project/nesi99999/riom/hsic_optimization/venv/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scale_wlg_persistent/filesets/project/nesi99999/riom/hsic_optimization/venv/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scale_wlg_persistent/filesets/project/nesi99999/riom/hsic_optimization/venv/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scale_wlg_persistent/filesets/project/nesi99999/riom/hsic_optimization/venv/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scale_wlg_persistent/filesets/project/nesi99999/riom/hsic_optimization/venv/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scale_wlg_persistent/filesets/project/nesi99999/riom/hsic_optimization/venv/include']
lapac